In [ ]:
acoes = ["PETR3.SA", "VALE3.SA", "EMBR3.SA"]
acoes

In [ ]:
import pandas_datareader.data as web
import yfinance as yf

In [ ]:
yf.pdr_override()
cotacoes = web.get_data_yahoo(acoes, start="2023-01-01", 
                              end="2024-05-09", threads=1)['Adj Close']
cotacoes

In [ ]:
# cotacoes.pct_change()
# cotacoes.pct_change().dropna()
variacoes = cotacoes.pct_change().dropna()
variacoes

In [ ]:
# variacoes.loc[:, :]
# variacoes.loc[:, ["PETR3.SA", "VALE3.SA", "EMBR3.SA"]]
variacoes = variacoes.loc[:, acoes]
variacoes

In [ ]:
# variacoes.mean(axis=0)
medias = variacoes.mean(axis=0)
medias

In [ ]:
# variacoes.cov()
matriz_covariancia = variacoes.cov()
matriz_covariancia

In [ ]:
cogito_ergo_sum_retorno = lambda carteira, medias: \
        2 ** medias.dot(carteira)

cogito_ergo_sum_risco = lambda carteira, matriz_covariancia: \
        abs(carteira.dot(matriz_covariancia).dot(carteira))

cogito_ergo_sum_fitness = lambda retorno, risco: retorno / risco

In [ ]:
type(cogito_ergo_sum_retorno)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
carteira = pd.Series(data=[1, 0, 0], index=acoes)
carteira

In [ ]:
carteira.iloc[0]

In [ ]:
retorno = cogito_ergo_sum_retorno(carteira, medias)
retorno

In [ ]:
risco = cogito_ergo_sum_risco(carteira, matriz_covariancia)
risco

In [ ]:
fitness = cogito_ergo_sum_fitness(retorno, risco)
fitness

In [ ]:
passo = 1

retornos = []
riscos = []
carteiras = []
fitnesses = []

colecao_petr = range(0, 101, passo)
for perc_petr in colecao_petr:
    
    colecao_vale = range(0, 101 - perc_petr, passo)
    for perc_vale in colecao_vale:

        perc_embr = 100 - perc_petr - perc_vale

        carteira = pd.Series(data=[perc_petr, perc_vale, perc_embr],
                             index=acoes)
        
        retorno = cogito_ergo_sum_retorno(carteira, medias)
        risco = cogito_ergo_sum_risco(carteira, matriz_covariancia)
        fitness = cogito_ergo_sum_fitness(retorno, risco)

        retornos.append(retorno)
        riscos.append(risco)
        carteiras.append(carteira)
        fitnesses.append(fitness)

In [ ]:
len(carteiras)

In [ ]:
fitnesses

In [ ]:
retornos = pd.Series(data=retornos)
riscos = pd.Series(data=riscos)
fitnesses = pd.Series(data=fitnesses)

In [ ]:
type(retornos)

In [ ]:
type(riscos)

In [ ]:
type(fitnesses)

In [ ]:
fitnesses.max()

In [ ]:
# fitnesses.idxmax()
indice_maior_fitness = fitnesses.idxmax()
indice_maior_fitness

In [ ]:
indice_maior_retorno = retornos.idxmax()
indice_maior_retorno

In [ ]:
indice_menor_risco = riscos.idxmin()
indice_menor_risco

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
grafico = plt.scatter(riscos, retornos, cmap="RdYlGn", c=fitnesses)

risco_minimo = riscos[indice_menor_risco]
retorno_do_risco_minimo = retornos[indice_menor_risco]
plt.scatter(risco_minimo, retorno_do_risco_minimo, color="green", edgecolors="black")

retorno_maximo = retornos[indice_maior_retorno]
risco_do_retorno_maximo = riscos[indice_maior_retorno]
plt.scatter(risco_do_retorno_maximo, retorno_maximo, color="red", edgecolors="black")

retorno_do_fitness_maximo = retornos[indice_maior_fitness]
risco_do_fitness_maximo = riscos[indice_maior_fitness]
plt.scatter(risco_do_fitness_maximo, retorno_do_fitness_maximo, 
            color="yellow", edgecolors="black")

plt.legend(["Carteiras",
            "Carteira Menor Risco",
            "Carteira Maior Retorno",
            "Carteira Maior Fitness"])

plt.title(label="Gráfico de Markowitz", fontsize=20, fontweight="bold",
          color="blue", style="italic")

plt.xlabel("Riscos", fontsize=15, fontweight="bold", color="red")
plt.ylabel("Retornos", fontsize=15, fontweight="bold", color="red",
           style="italic")

plt.colorbar(grafico, label="Fitness", orientation = "vertical", pad=0.02)

plt.show()

In [ ]:
carteiras[indice_maior_fitness]

In [ ]:
carteiras[indice_maior_retorno]

In [ ]:
carteiras[indice_menor_risco]

---
# Algoritmos Genéticos

In [ ]:
# vamos reaproveitar esta variavel
variacoes

In [ ]:
# vamos reaproveitar esse variável (constante)
medias

In [ ]:
# vamos reaproveitar essa variável (constante)
matriz_covariancia

In [ ]:
qtd_croms_populacao_geral = 40

In [ ]:
qtd_genes = len(acoes)
qtd_genes

In [ ]:
np.random.seed(42)
carteiras = np.random.randint(low=0, high=10, 
                            size=(qtd_croms_populacao_geral, qtd_genes))
carteiras

In [ ]:
from sklearn import preprocessing

In [ ]:
cromossomos = preprocessing.normalize(carteiras, norm="l1", axis=1)
cromossomos

In [ ]:
cromossomos = pd.DataFrame(data=cromossomos, columns=acoes)
cromossomos

In [ ]:
def ces_retornos(carteiras: pd.DataFrame, medias: pd.Series) -> pd.Series:
    """
    Esta função recebe multiplas carteiras e as médias periódicas das
    variações percentuais
    :param carteiras = N Carteiras (linhas) por M acoes (colunas)
    :param medias = Series com as médias das variacoes diárias das acoes

    :return todos os retornos das carteiras fornecidas de uma vez só!!!
    """
    return 2 ** carteiras.dot(medias)

In [ ]:
print(2 ** cogito_ergo_sum_retorno(cromossomos.iloc[0], medias))
print(2 ** cogito_ergo_sum_retorno(cromossomos.iloc[1], medias))
print(2 ** cogito_ergo_sum_retorno(cromossomos.iloc[2], medias))

In [ ]:
print(ces_retornos(cromossomos, medias))

In [ ]:
cromossomos["Retornos"] = ces_retornos(cromossomos, medias)

In [ ]:
cromossomos

In [ ]:
def ces_riscos(carteiras: pd.DataFrame, 
               matriz_covariancia: pd.DataFrame) -> pd.Series:
    """
    Esta função recebe multiplas carteiras e a matriz de covariâncias
    entre os ativos (genes)
    :param carteiras = N Carteiras (linhas) por M acoes (colunas)
    :param matriz_covariancia = matriz das covariancias entre os ativos

    :return todos os riscos das carteiras fornecidas de uma vez só!!!
    """
    return (carteiras.dot(matriz_covariancia) * carteiras).\
                                            sum(axis=1).__abs__()

In [ ]:
print(abs(cogito_ergo_sum_risco(cromossomos.loc[0, acoes],
                            matriz_covariancia)))
print(abs(cogito_ergo_sum_risco(cromossomos.loc[1, acoes],
                            matriz_covariancia)))
print(abs(cogito_ergo_sum_risco(cromossomos.loc[2, acoes],
                            matriz_covariancia)))

In [ ]:
print(ces_riscos(cromossomos.loc[:, acoes], matriz_covariancia))

In [ ]:
cromossomos["Riscos"] = ces_riscos(cromossomos.loc[:, acoes],
                                   matriz_covariancia)
cromossomos

In [ ]:
def ces_fitnesses(retornos: pd.Series, riscos: pd.Series) -> pd.Series:
    """
    Esta função recebe multiplos retornos (pd.series) e múltiplos 
    riscos (pd.series) e retorna multiplos fitnesses (pd.series)

    :param retornos = retornos dos cromossomos
    :param riscos = risco dos cromossomos

    :return todos os fitnesses dos cromossomos
    """
    return retornos / riscos

In [ ]:
cromossomos["Fitnesses"] = ces_fitnesses(cromossomos.loc[:, "Retornos"],
                                         cromossomos.loc[:, "Riscos"])
cromossomos

In [ ]:
np.random.seed(42)
# np.random.choice(cromossomos.index,
#                  size=6, replace=False)
indices_cromossomos_sorteados = np.random.choice(cromossomos.index,
                                                size=6, replace=False)
indices_cromossomos_sorteados

In [ ]:
# cromossomos
# cromossomos.loc[[3, 5, 8]]
# cromossomos.loc[indices_cromossomos_sorteados]
cromossomos_sorteados = cromossomos.loc[indices_cromossomos_sorteados]
cromossomos_sorteados

In [ ]:
# cromossomos_sorteados.loc[:, "Fitnesses"]
# cromossomos_sorteados.loc[:, "Fitnesses"] / \
#     cromossomos_sorteados.loc[:, "Fitnesses"].sum()
percentagens_relativas_fitnesses = \
    cromossomos_sorteados.loc[:, "Fitnesses"] / \
        cromossomos_sorteados.loc[:, "Fitnesses"].sum()
percentagens_relativas_fitnesses

In [ ]:
# percentagens_relativas_fitnesses
# percentagens_relativas_fitnesses.cumsum()
percentagens_acumuladas_fitnesses = \
    percentagens_relativas_fitnesses.cumsum()
percentagens_acumuladas_fitnesses

In [ ]:
np.random.seed(42)
al = np.random.rand()
al

In [ ]:
# al > percentagens_acumuladas_fitnesses
posicao_cromossomo_sorteado = \
    (al > percentagens_acumuladas_fitnesses).sum()
posicao_cromossomo_sorteado

In [ ]:
cromossomo_pai = cromossomos_sorteados.iloc[posicao_cromossomo_sorteado]
cromossomo_pai

In [ ]:
np.random.seed(44)
al = np.random.rand()
al

In [ ]:
percentagens_acumuladas_fitnesses

In [ ]:
# al > percentagens_acumuladas_fitnesses
# (al > percentagens_acumuladas_fitnesses).sum()
posicao_cromossomo_sorteado = \
    (al > percentagens_acumuladas_fitnesses).sum()
posicao_cromossomo_sorteado

In [ ]:
cromossomo_mae = cromossomos_sorteados.iloc[posicao_cromossomo_sorteado]
cromossomo_mae

In [ ]:
np.random.seed(45)
al = np.random.rand()
al

In [ ]:
parte_genes_pai = al * cromossomo_pai.loc[acoes]
parte_genes_pai

In [ ]:
parte_genes_mae = (1 - al) * cromossomo_mae.loc[acoes]
parte_genes_mae

In [ ]:
cromossomo_filho = parte_genes_mae + parte_genes_pai
cromossomo_filho

In [ ]:
np.random.seed(46)
al = np.random.rand()
al

In [ ]:
parte_genes_pai = al * cromossomo_pai.loc[acoes]
parte_genes_pai

In [ ]:
parte_genes_mae = (1 - al) * cromossomo_mae.loc[acoes]
parte_genes_mae

In [ ]:
cromossomo_filho_dois = parte_genes_mae + parte_genes_pai
cromossomo_filho_dois

In [ ]:
cromossomo_filho.sum()

In [ ]:
cromossomo_filho_dois.sum()

In [ ]:
# np.random.choice(acoes, size=2, replace=False)
np.random.seed(47)
genes_sorteados = np.random.choice(acoes, size=2, replace=False)
genes_sorteados

In [ ]:
# cromossomo_filho_um
# cromossomo_filho_um.loc[genes_sorteados]
cromossomo_filho.loc[genes_sorteados]

In [ ]:
cromossomo_filho.loc[genes_sorteados].iloc[::-1]

In [ ]:
mutante = cromossomo_filho.copy()
mutante.loc[genes_sorteados] = \
    cromossomo_filho.loc[genes_sorteados].iloc[::-1].values
mutante

In [ ]:
np.random.seed(48)
genes_sorteados = np.random.choice(acoes, size=2, replace=False)
genes_sorteados

In [ ]:
mutante_dois = cromossomo_filho_dois.copy()
mutante_dois.loc[genes_sorteados] = \
    cromossomo_filho_dois.loc[genes_sorteados].iloc[::-1].values
mutante_dois

In [ ]:
np.random.seed(49)
genes_sorteados = np.random.choice(acoes, size=2, replace=False)
genes_sorteados

In [ ]:
soma_genes = cromossomo_filho.loc[genes_sorteados].sum()
soma_genes

In [ ]:
mutante_a = cromossomo_filho.copy()
mutante_a.loc[genes_sorteados[0]] = soma_genes
mutante_a.loc[genes_sorteados[1]] = 0

mutante_b = cromossomo_filho.copy()
mutante_b.loc[genes_sorteados[0]] = 0
mutante_b.loc[genes_sorteados[1]] = soma_genes

In [ ]:
print(cromossomo_filho)
print("-" * 100)
print(mutante_a)
print("-" * 100)
print(mutante_b)

In [ ]:
np.random.seed(50)
genes_sorteados = np.random.choice(acoes, size=2, replace=False)
genes_sorteados

In [ ]:
soma_genes = cromossomo_filho_dois.loc[genes_sorteados].sum()
soma_genes

In [ ]:
mutante_cinco = cromossomo_filho_dois.copy()
mutante_cinco.loc[genes_sorteados[0]] = soma_genes
mutante_cinco.loc[genes_sorteados[1]] = 0

mutante_seis = cromossomo_filho_dois.copy()
mutante_seis.loc[genes_sorteados[0]] = 0
mutante_seis.loc[genes_sorteados[1]] = soma_genes

In [ ]:
print(cromossomo_filho_dois)
print("-" * 100)
print(mutante_cinco)
print("-" * 100)
print(mutante_seis)

In [ ]:
df_nova_geracao = pd.DataFrame(data=[cromossomo_filho, cromossomo_filho_dois,
                                        mutante, mutante_dois,
                                        mutante_a, mutante_b,
                                        mutante_cinco, mutante_seis])
df_nova_geracao

In [ ]:
df_nova_geracao["Retornos"] = \
    ces_retornos(carteiras=df_nova_geracao, medias=medias)
df_nova_geracao

In [ ]:
df_nova_geracao["Riscos"] = ces_riscos(
    carteiras=df_nova_geracao.loc[:, acoes], 
    matriz_covariancia=matriz_covariancia)

df_nova_geracao

In [ ]:
df_nova_geracao["Fitnesses"] = ces_fitnesses(
    retornos=df_nova_geracao.loc[:, "Retornos"],
    riscos=df_nova_geracao.loc[:, "Riscos"]
)
df_nova_geracao

In [ ]:
cromossomos_sorteados

In [ ]:
# cromossomos_sorteados
# cromossomos_sorteados["Fitnesses"]
# cromossomos_sorteados["Fitnesses"].idxmin()
indice_cromossomo_ruim = cromossomos_sorteados["Fitnesses"].idxmin()
indice_cromossomo_ruim

In [ ]:
indice_cromossomo_bom = df_nova_geracao["Fitnesses"].idxmax()
indice_cromossomo_bom

In [ ]:
cromossomos_sorteados.loc[indice_cromossomo_ruim] = \
    df_nova_geracao.loc[indice_cromossomo_bom].values

cromossomos_sorteados

---

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn import preprocessing
from datetime import datetime, timedelta

In [2]:
def ces_retornos(carteiras: pd.DataFrame, medias: pd.Series) -> pd.Series:
    """
    Esta função recebe multiplas carteiras e as médias periódicas das
    variações percentuais
    :param carteiras = N Carteiras (linhas) por M acoes (colunas)
    :param medias = Series com as médias das variacoes diárias das acoes

    :return todos os retornos das carteiras fornecidas de uma vez só!!!
    """
    return 2 ** carteiras.dot(medias)

def ces_riscos(carteiras: pd.DataFrame, 
               matriz_covariancia: pd.DataFrame) -> pd.Series:
    """
    Esta função recebe multiplas carteiras e a matriz de covariâncias
    entre os ativos (genes)
    :param carteiras = N Carteiras (linhas) por M acoes (colunas)
    :param matriz_covariancia = matriz das covariancias entre os ativos

    :return todos os riscos das carteiras fornecidas de uma vez só!!!
    """
    return (carteiras.dot(matriz_covariancia) * carteiras).\
                                            sum(axis=1).__abs__()

def ces_fitnesses(retornos: pd.Series, riscos: pd.Series) -> pd.Series:
    """
    Esta função recebe multiplos retornos (pd.series) e múltiplos 
    riscos (pd.series) e retorna multiplos fitnesses (pd.series)

    :param retornos = retornos dos cromossomos
    :param riscos = risco dos cromossomos

    :return todos os fitnesses dos cromossomos
    """
    return retornos / riscos

In [3]:
def gerar_nova_geracao(acoes, medias, matriz_covariancia, cromossomo_filho_um, 
                       cromossomo_filho_dois, mutante_um, mutante_dois, mutante_tres, 
                       mutante_quatro, mutante_cinco, mutante_seis):
    
    df_nova_geracao = pd.DataFrame(data=[cromossomo_filho_um, cromossomo_filho_dois,
                                                    mutante_um, mutante_dois,
                                                    mutante_tres, mutante_quatro,
                                                    mutante_cinco, mutante_seis])
            
    df_nova_geracao["Retornos"] = \
                ces_retornos(carteiras=df_nova_geracao, medias=medias)

    df_nova_geracao["Riscos"] = ces_riscos(
                carteiras=df_nova_geracao.loc[:, acoes], 
                matriz_covariancia=matriz_covariancia)

    df_nova_geracao["Fitnesses"] = ces_fitnesses(
                retornos=df_nova_geracao.loc[:, "Retornos"],
                riscos=df_nova_geracao.loc[:, "Riscos"]
            )
    
    return df_nova_geracao


def mutacao_dois(acoes, cromossomo_filho):
    genes_sorteados = np.random.choice(acoes, size=2, replace=False)
    soma_genes = cromossomo_filho.loc[genes_sorteados].sum()
    mutante_a = cromossomo_filho.copy()
    mutante_a.loc[genes_sorteados[0]] = soma_genes
    mutante_a.loc[genes_sorteados[1]] = 0

    mutante_b = cromossomo_filho.copy()
    mutante_b.loc[genes_sorteados[0]] = 0
    mutante_b.loc[genes_sorteados[1]] = soma_genes
    return mutante_a,mutante_b

def mutacao_um(acoes, cromossomo_filho):
    genes_sorteados = np.random.choice(acoes, size=2, replace=False)
    mutante = cromossomo_filho.copy()
    mutante.loc[genes_sorteados] = \
                cromossomo_filho.loc[genes_sorteados].iloc[::-1].values
        
    return mutante

def crossover(acoes, cromossomo_pai, cromossomo_mae):
    al = np.random.rand()
    parte_genes_pai = al * cromossomo_pai.loc[acoes]
    parte_genes_mae = (1 - al) * cromossomo_mae.loc[acoes]
    cromossomo_filho_um = parte_genes_mae + parte_genes_pai
    return cromossomo_filho_um

def roda_do_acaso(cromossomos_sorteados):
    percentagens_relativas_fitnesses = \
                cromossomos_sorteados.loc[:, "Fitnesses"] / \
                    cromossomos_sorteados.loc[:, "Fitnesses"].sum()

            # gera um series com as percentagens acumuladas
            # 0.2, 0.65, 0.70, 0.80, 0.95, 1.00
    percentagens_acumuladas_fitnesses = \
                percentagens_relativas_fitnesses.cumsum()
            
            # esse comando gera um aleatorio de 0 até 1
            # ex. 0.68
    al = np.random.rand()

            # retorna a posição do cromossomo sorteado
            # no exemplo acima seria o cromossomo de posição
            # 2 (terceiro cromossomo)
    posicao_cromossomo_sorteado = \
                (al > percentagens_acumuladas_fitnesses).sum()

    cromossomo_pai = cromossomos_sorteados.iloc[posicao_cromossomo_sorteado]

    cromossomo_mae = cromossomo_pai.copy()

    while (cromossomo_mae == cromossomo_pai).all():
        al = np.random.rand()
        posicao_cromossomo_sorteado = \
                    (al > percentagens_acumuladas_fitnesses).sum()

        cromossomo_mae = cromossomos_sorteados.iloc[posicao_cromossomo_sorteado]
    return cromossomo_pai,cromossomo_mae

def gerar_cromossomos_base(qtd_croms_populacao_geral, acoes, medias, matriz_covariancia):
    qtd_genes = len(acoes)

    carteiras = np.random.randint(low=0, high=10, 
                                size=(qtd_croms_populacao_geral, qtd_genes))
    cromossomos = preprocessing.normalize(carteiras, norm="l1", axis=1)
    cromossomos = pd.DataFrame(data=cromossomos, columns=acoes)

    cromossomos["Retornos"] = ces_retornos(cromossomos, medias)
    cromossomos["Riscos"] = ces_riscos(cromossomos.loc[:, acoes],
                                    matriz_covariancia)
    cromossomos["Fitnesses"] = ces_fitnesses(cromossomos.loc[:, "Retornos"],
                                            cromossomos.loc[:, "Riscos"])
                                            
    return cromossomos

In [4]:
def gerar_variacoes_acoes(acoes, data_inicio, data_fim):
    cotacoes = yf.download(tickers=acoes, start=data_inicio, end=data_fim)["Adj Close"]
    variacoes = cotacoes.pct_change().dropna().loc[:, acoes]

    return variacoes

In [5]:
def moneta_ag(variacoes: pd.DataFrame, 
              qtd_iteracoes = 10, qtd_epocas = 40, qtd_croms_populacao_geral = 40):

    acoes = variacoes.columns

    medias = variacoes.mean(axis=0)
    matriz_covariancia = variacoes.cov()

    cromossomos = gerar_cromossomos_base(qtd_croms_populacao_geral, acoes, medias, 
                                         matriz_covariancia)

    for _ in range(qtd_epocas):
        indices_cromossomos_sorteados = np.random.choice(cromossomos.index,
                                                    size=6, replace=False)
        cromossomos_sorteados = cromossomos.loc[indices_cromossomos_sorteados]

        for _ in range(qtd_iteracoes):

            # RODA DO ACASO -------------------------------------
            # gera um series com as percentagens relativas:
            # 0.2, 0.45, 0.05, 0.10, 0.15, 0.05
            cromossomo_pai, cromossomo_mae = roda_do_acaso(cromossomos_sorteados)
            
            # RODA DO ACASO -------------------------------------

            # CROSSOVER -----------------------------------------

            cromossomo_filho_um = crossover(acoes, cromossomo_pai, cromossomo_mae)
            cromossomo_filho_dois = crossover(acoes, cromossomo_pai, cromossomo_mae)

            # CROSSOVER -----------------------------------------

            # MUTAÇÃO DO TIPO 1 ---------------------------------
            mutante_um = mutacao_um(acoes, cromossomo_filho_um)
            mutante_dois = mutacao_um(acoes, cromossomo_filho_dois)
            # MUTAÇÃO DO TIPO 1 ---------------------------------

            # MUTAÇÃO DO TIPO 2 ---------------------------------
            mutante_tres, mutante_quatro = mutacao_dois(acoes, cromossomo_filho_um)
            mutante_cinco, mutante_seis = mutacao_dois(acoes, cromossomo_filho_dois)
            # MUTAÇÃO DO TIPO 2 --------------------------------


            df_nova_geracao = gerar_nova_geracao(acoes, medias, matriz_covariancia, 
                                                 cromossomo_filho_um, cromossomo_filho_dois, 
                                                 mutante_um, mutante_dois, mutante_tres, 
                                                 mutante_quatro, mutante_cinco, mutante_seis)

            nome_cromossomo_ruim = cromossomos_sorteados["Fitnesses"].idxmin()

            nome_cromossomo_bom = df_nova_geracao["Fitnesses"].idxmax()

            fitness_pior_pai = cromossomos_sorteados.loc[nome_cromossomo_ruim].loc["Fitnesses"]
            fitness_melhor_filho = df_nova_geracao.loc[nome_cromossomo_bom].loc["Fitnesses"]

            if fitness_melhor_filho > fitness_pior_pai:
                cromossomos_sorteados.loc[nome_cromossomo_ruim] = \
                    df_nova_geracao.loc[nome_cromossomo_bom].values
        
        cromossomos.loc[indices_cromossomos_sorteados] = \
            cromossomos_sorteados.values

    indice_melhor_cromossomo = cromossomos["Fitnesses"].idxmax()
    melhor_cromossomo = cromossomos.loc[indice_melhor_cromossomo]

    return melhor_cromossomo

In [7]:
acoes = ['PETR3.SA', 'VALE3.SA', 'EMBR3.SA', 'ABEV3.SA', 'MGLU3.SA']
variacoes = gerar_variacoes_acoes(acoes=acoes, data_inicio="2020-01-01",
                                  data_fim="2024-06-09")

carteira_otima = moneta_ag(variacoes=variacoes,
                            qtd_iteracoes=10, 
                            qtd_epocas=40, 
                            qtd_croms_populacao_geral=40)

[*********************100%%**********************]  4 of 5 completed

5 Failed downloads:
['VALE3.SA', 'EMBR3.SA', 'MGLU3.SA', 'ABEV3.SA', 'PETR3.SA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1577847600&period2=1717902000&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=QNOhS53oGQG (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))"))
g:\Meu Drive\Modelos\cpp_aula\python_dois\moneta\venv_moneta\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
g:\Meu Drive\Modelos\cpp_aula\python_dois\moneta\venv_moneta\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
g:\Meu Drive\Modelos\cpp_aula\python_dois\moneta\venv_moneta\Lib\site-packages\p

KeyError: nan

In [ ]:
carteira_otima

In [ ]:
carteira_otima.loc[acoes].sum()

In [ ]:
simbolos = \
{"US": ['^GSPC', 
               'AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 
               'META', 'TSLA', 'BRK-B', 'V', 'JNJ', 
               'JPM', 'WMT', 'NVDA', 'UNH', 'MA', 
               'PG', 'HD', 'DIS', 'BAC', 'PYPL', 
               'CMCSA', 'VZ', 'ADBE', 'NFLX', 'KO', 
               'XOM', 'PEP', 'T', 'PFE', 'INTC', 
               'MRK', 'CSCO', 'ABT', 'CRM', 'CVX', 
               'ABBV', 'TMO', 'ACN', 'NKE', 'AVGO', 
               'MCD', 'COST', 'QCOM', 'NEE', 'TXN', 
               'DHR', 'MDT', 'HON', 'UNP', 'LIN', 
               'AMGN', 'PM', 'SBUX', 'LLY', 'UPS', 
               'LOW', 'AMT', 'IBM', 'CAT', 'GILD', 
               'MMM', 'MO', 'GE', 'CHTR', 'NOW', 
               'BLK', 'INTU', 'SPGI', 'ISRG', 'AMD', 
               'CVS', 'ZTS', 'PLD', 'AXP', 'TGT', 
               'FIS', 'APD',  'MDLZ', 'ADP', 'LMT', 
               'CME', 'DUK', 'CL', 'ICE', 'BDX', 
               'CB', 'SPG', 'CI', 'NSC', 'VRTX', 
               'CCI', 'RTX', 'TMUS', 'BKNG', 'DE', 
               'TJX', 'SYK', 'TFC'],

"BR": ['BOVA11.SA', 
               'ABEV3.SA', 'AGRO3.SA', 'ALPA3.SA', 'ALOS3.SA', 'ALUP11.SA', 
               'ANIM3.SA', 'ARZZ3.SA', 'B3SA3.SA', 'BAZA3.SA', 'BBAS3.SA', 
               'BBDC3.SA', 'BBSE3.SA', 'BEEF3.SA', 'BEES3.SA', 'BHIA3.SA', 
               'BMEB3.SA', 'BNBR3.SA', 'BRAP3.SA', 'BRFS3.SA', 'BRKM3.SA', 
               'BRSR3.SA', 'CCRO3.SA', 'CEEB3.SA', 'CGAS3.SA', 'CIEL3.SA', 
               'CLSC3.SA', 'CMIG3.SA', 'COCE3.SA', 'COGN3.SA', 'CPFE3.SA', 
               'CPLE3.SA', 'CSAN3.SA', 'CSMG3.SA', 'CSNA3.SA', 'CVCB3.SA', 
               'CYRE3.SA', 'DASA3.SA', 'DIRR3.SA', 'DXCO3.SA', 'ECOR3.SA', 
               'EGIE3.SA', 'ELET3.SA', 'EMBR3.SA', 'ENAT3.SA', 'ENEV3.SA', 
               'ENGI11.SA', 'ENGI3.SA', 'EQPA3.SA', 'EQTL3.SA', 'EVEN3.SA', 
               'EZTC3.SA', 'FLRY3.SA', 'FRAS3.SA', 'GGBR3.SA', 'GOAU3.SA', 
               'GRND3.SA', 'GUAR3.SA', 'HYPE3.SA', 'ITSA3.SA', 'ITUB3.SA', 
               'JBSS3.SA', 'JHSF3.SA', 'KEPL3.SA', 'KLBN11.SA', 'KLBN3.SA', 
               'LEVE3.SA', 'LIGT3.SA', 'LOGN3.SA', 'LREN3.SA', 'MDIA3.SA', 
               'MGLU3.SA', 'MILS3.SA', 'MOAR3.SA', 'MRFG3.SA', 'MRVE3.SA', 
               'MULT3.SA', 'MYPK3.SA', 'ODPV3.SA', 'OFSA3.SA', 'PETR3.SA', 
               'PNVL3.SA', 'POMO3.SA', 'POSI3.SA', 'PRIO3.SA', 'PSSA3.SA', 
               'RADL3.SA', 'RAIL3.SA', 'RANI3.SA', 'RAPT3.SA', 'RENT3.SA', 
               'ROMI3.SA', 'SANB11.SA', 'SANB3.SA', 'SBSP3.SA', 'SCAR3.SA', 
               'SLCE3.SA', 'SMTO3.SA', 'TAEE11.SA', 'TASA3.SA', 'TEND3.SA', 
               'TGMA3.SA', 'TIMS3.SA', 'TOTS3.SA', 'TRPL3.SA', 'TUPY3.SA', 
               'UGPA3.SA', 'UNIP3.SA', 'USIM3.SA', 'VALE3.SA', 'VIVT3.SA', 
               'VLID3.SA', 'VULC3.SA', 'WEGE3.SA', 'WHRL3.SA', 'YDUQ3.SA']}

In [ ]:
def busca_cotacoes(simbolos: list, cotacoes_anteriores: int, intervalo: str) -> pd.DataFrame:

    """
    Função que busca as variações periódicas das ações

    Args:
    simbolos (list): Lista com os símbolos (tickers) das ações
    cotacoes_anteriores (int): Quantidade de cotações anteriores a serem buscadas para as variações das ações
    intervalo (str): Intervalo de busca das variações periódicas das ações. 'd' para diário, 'w' para semanal
    maiores_medias (int): Quantidade de ações com maiores médias de retorno a serem filtradas

    Returns:
    variacoes (pd.DataFrame): DataFrame com as variações periódicas das ações
    """

    # data de hoje (formato datetime)
    hoje_dtm: datetime = datetime.today()

    # data de hoje (formato string: aaaa-mm-dd). Será usado como data final dos dados a serem buscados
    hoje_str: str = hoje_dtm.strftime('%Y-%m-%d')

    # data de início da busca (data de hoje menos a quantidade de cotações anteriores)
    if intervalo == "d":
        # se o intervalo for diário, subtrai a quantidade de dias
        inicio: datetime = hoje_dtm - timedelta(days=cotacoes_anteriores)
    elif intervalo == "w":
        # se o intervalo for semanal, subtrai a quantidade de semanas
        inicio: datetime = hoje_dtm - timedelta(weeks=cotacoes_anteriores)
    
    # converte a data de início para string (aaaa-mm-dd)
    inicio: str = inicio.strftime('%Y-%m-%d')

    # busca as cotações das ações para o intervalo especificado
    cotacoes: pd.DataFrame = yf.download(simbolos, start=inicio, end=hoje_str)['Adj Close']

    return cotacoes

In [ ]:
def formata_cotacoes(cotacoes: pd.DataFrame, intervalo: str, maiores_medias: int) -> pd.DataFrame:

    """
    Função que formata as cotações das ações para variações periódicas e filtra as ações com maiores médias de retorno

    Args:
    cotacoes (pd.DataFrame): DataFrame com as cotações das ações
    intervalo (str): Intervalo de busca das variações periódicas das ações. 'd' para diário, 'w' para semanal
    maiores_medias (int): Quantidade de ações com maiores médias de retorno a serem filtradas

    Returns:
    variacoes_intervaladas_filtradas (pd.DataFrame): DataFrame com as variações periódicas das ações filtradas
    """

    # elimina as colunas (axis = 1: nome das ações) que possuem valores nulos para datas específicas dentro do intervalo de busca    
    cotacoes.dropna(axis=1, inplace=True)

    # calcula as variações diárias das ações e elimina as linhas com valores nulos.
    # valores nulos podem ocorrer quando a ação não possui cotação em um determinado dia
    variacoes: pd.DataFrame = cotacoes.pct_change().dropna()

    # filtra as variações periódicas das ações (a cada 5 dias ou todos os dias)
    variacoes_intervaladas: pd.DataFrame = variacoes.iloc[::5] \
                                            if intervalo == "w" else variacoes

    if maiores_medias > 0:
        # filtra as maiores médias de retorno pelo intervalo escolhido
        # variacoes_intervaladas_filtradas = filtra_maiores_medias(variacoes_intervaladas, n=maiores_medias)

        # calcula as médias dos retornos das ações
        medias: pd.Series = variacoes_intervaladas.mean(axis=0)

        # o método 'nlargest' está presente em qualquer objeto do tipo 'Series'. Esse método retorna outro 'Series' com os 'n' maiores valores
        acoes_maiores_medias: pd.Series = medias.nlargest(maiores_medias)

        # pega as ações com as maiores médias de retorno
        variacoes_intervaladas_filtradas: pd.DataFrame = \
            variacoes_intervaladas.loc[:, acoes_maiores_medias.index]

        return variacoes_intervaladas_filtradas
    
    return variacoes_intervaladas

In [ ]:
pais = "BR"
cotacoes_anteriores = 200
intervalo = "d"
maiores_medias = 10

cotacoes = busca_cotacoes(simbolos=acoes,# simbolos[pais][1:5], 
                          cotacoes_anteriores=cotacoes_anteriores, 
                          intervalo=intervalo)

variacoes = formata_cotacoes(cotacoes=cotacoes, 
                             intervalo=intervalo, 
                             maiores_medias=maiores_medias)

In [ ]:
variacoes.head()

In [ ]:
variacoes.tail()

In [ ]:
carteira_otima = moneta_ag(variacoes=variacoes)
carteira_otima

In [ ]:
def arredonda_para_baixo(numero, casas_decimais=0):
    multiplicador = 10 ** casas_decimais
    return int(numero * multiplicador) / multiplicador

def gera_df_carteira(carteira_final: pd.Series, cotacoes: pd.DataFrame, pais: str,
                     percentual_filtrar: int = 5, valor_investir: float = 10000):
    
    """
    Função que gera um DataFrame com as informações da carteira final

    Args:
    carteira_final (pd.Series): Carteira final com os percentuais das ações
    simbolos (list): Lista com os símbolos (tickers) das ações
    cotacoes (pd.DataFrame): DataFrame com as cotações das ações
    percentual_filtrar (int): Percentual mínimo para filtrar as ações da carteira
    valor_investir (float): Valor a ser investido na carteira

    Returns:
    df_carteira (pd.DataFrame): DataFrame com as informações da carteira final

    """

    # pega os símbolos das ações
    simbolos = carteira_final.loc[~carteira_final.index.isin(["Retornos", "Riscos", "Fitnesses"])].index

    # ordena as ações com maiores percentuais na carteira
    carteira_final = carteira_final.loc[simbolos].sort_values(ascending=False)

    # filtra as ações com percentuais maiores que o percentual mínimo
    carteira_final_filtrada = carteira_final.loc[carteira_final.values > percentual_filtrar / 100]

    # se todas as ações da carteira, após o filtro, tiverem percentuais menores que o percentual mínimo, retorna um DataFrame vazio
    if carteira_final_filtrada.empty:
        return None

    # pega os símbolos das ações após o filtro
    simbolos_filtrados = carteira_final_filtrada.index

    # ultimos valores das ações que passaram pelo filtro
    ultimos_precos = cotacoes.loc[:, simbolos_filtrados].iloc[-1]

    # quantidade de ações a serem compradas para cada ação da carteira já filtrada
    qtd_acoes = carteira_final_filtrada * valor_investir / ultimos_precos

    # a quantidade de ações precisa ser filtrada para valores inteiros (mercado BR) e valores com 6 casas decimais (mercado US)
    qtd_acoes_ajustado = \
    pd.Series(map(lambda perc: arredonda_para_baixo(numero=perc, 
                                                    casas_decimais=0 if pais == "BR" else 6), 
                                                    qtd_acoes), 
                                                    index=simbolos_filtrados)
    
    # cria o DataFrame com as informações da carteira final
    df_carteira = (carteira_final_filtrada * 100).round(2).to_frame(name="Investido (%)")

    # cria a coluna 'Qtd de Acoes' para cada ação no DataFrame da carteira
    df_carteira.loc[:, 'Qtd de Acoes'] = qtd_acoes_ajustado.values

    # cria a coluna 'Investido (R$ ou US$)' para cada ação no DataFrame da carteira
    df_carteira.loc[:, f"Investido ({'R$' if pais == 'BR' else 'US$'})"] = (ultimos_precos * df_carteira.loc[:, 'Qtd de Acoes']).round(2)

    # insere a coluna 'Precos (R$ ou US$)' pada cada ação no DataFrame da carteira
    df_carteira.insert(0, f"Precos ({'R$' if pais == 'BR' else 'US$'})", ultimos_precos.round(2))

    return df_carteira

In [ ]:
df_carteira = gera_df_carteira(carteira_final=carteira_otima, cotacoes=cotacoes, pais=pais, percentual_filtrar=1, valor_investir=10000)

In [ ]:
df_carteira